# Shocks and grids

In [1]:
using Parameters

In [ ]:
function fun_shocks_grid(ae)
    
    @unpack newΛ, σEps, λ, NNN, MMM  = ae #midshock,
    @unpack Ny, ϕ, ω, Λ, grid_yc, grid_yd = ae  # shock grid bc_grid, 
    @unpack M, SDFC, rf, α, ρshock, ϑ, σshock,q = ae # SDF
    @unpack δ, α0,b_inf, b_sup,Nb,bs_inf, bs_sup, Nbs = ae
    @unpack σcop, ρcop, Nc, crash = ae #Commodity shock
    
    #Initial value for q
    ae.q .= ones(Nb, Ny, Nbs, Nc) / (1+rf)     
    
    # Discretize productivity (a) shock 
    
    Λ, YYY = fun_tauchen(; σEps=σEps, λ=λ, NNN=NNN, MMM=MMM)
    ae.Λ .=  Λ
    sizeΛ = size(Λ) 
    sizeΛ = sizeΛ[2] 
    ae.grid_yc .= exp.(YYY) 
    
    longΛ = Λ^100  # unconditional distribution from Markov matrix
    Λunc = similar(Λ)
    suΛunc = similar(Λ)
    
    
    #Cumulative probability conditional on the shock today, and long-run unconditional probability vector.
    for nnn = 1:sizeΛ
        ae.newΛ[nnn] = Λ[nnn]
        Λunc[nnn] = longΛ[1,nnn] 
        suΛunc[nnn] = sum(Λunc[1:nnn])
        for bbb = 2:sizeΛ 
          ae.newΛ[nnn,bbb] = ae.newΛ[nnn,bbb-1] + Λ[nnn,bbb]  
        end 
    end
    
    
    # Discretize commodity (x) shock 
    
    Γ, YYY = fun_tauchen(; σEps=σcop , λ=ρcop, NNN=Nc, MMM=1.50)
    ae.Γ .= Γ
    size_Γ = size(Γ) 
    size_Γ = size_Γ[2]
    ae.grid_c .= exp.(YYY) 
    #model sharp drops in the price of copper at the highest price of copper
    ae.Γ[size_Γ, size_Γ]  = max(ae.Γ[size_Γ,size_Γ] - crash, 0) 
    ae.Γ[size_Γ, size_Γ-1]  = min(ae.Γ[size_Γ, size_Γ-1] + crash,1)

    #at the second highes price of copper
    ae.Γ[size_Γ-1, size_Γ-1] = max(ae.Γ[size_Γ-1, size_Γ-1] - crash,0) 
    ae.Γ[size_Γ-1, size_Γ-2] = min(ae.Γ[size_Γ-1, size_Γ-2] + crash,1)

    longΓ = ae.Γ^100  #unconditional distribution from Markov matrix
        
    #Cumulative probability conditional on the shock today, and long-run unconditional probability vector.
    Λunccop = similar(Γ)
    suΛunccop = similar(Γ)
    for nnn = 1:size_Γ
        ae.newΓ[nnn] = ae.Γ[nnn]  
        Λunccop[nnn] = longΓ[1,nnn]  
        suΛunccop[nnn] = sum(Λunccop[1:nnn])
        for bbb = 2:size_Γ
            ae.newΓ[nnn,bbb] = ae.newΓ[nnn, bbb-1] + ae.Γ[nnn,bbb]  
        end 
    end    
    
    # Shock credit and default grid .......
    sizeΛ = size(Λ)[2] 
    midshock = Int((sizeΛ+1)/2)    
    
    for a = 1:Ny 
        ae.grid_yc[a] = grid_yc[a] 
        ae.grid_yd[a] = min(ϕ*(ae.grid_yc[Int(midshock)]), ae.grid_yc[a]) - ω # CHECK THIS SO IT CAN BE SIMPLIFIED  
    end    

    # SDF grid .......
    for a = 1:Ny ;
        for z = 1:Ny 
             ae.M[z,a] = (1-SDFC) + SDFC*exp( - (α0 + δ*log(grid_yc[a]))*(log(grid_yc[z])-
                         ρshock*log(grid_yc[a])) - (1/2)*((α0 + δ*log(grid_yc[a]))^2)*(σshock^2) )  
         end 
    end 
    
end



## Aux functions

In [ ]:
function fun_tauchen(; 
                        σEps = 0.0064,
                        λ    = 0.597,
                        NNN  = 5,
                        MMM  = 1.95)
    
    σ2Eps = σEps^2
    σY = ( σ2Eps / (1-(λ^2)) )^(0.5)
    YYY = zeros(NNN)
    YYY[NNN] = MMM*σY
    YYY[1] = (-1)*YYY[NNN]
    WWW = (YYY[NNN] - YYY[1])/(NNN-1)
    
    for n = 2:NNN-1
        YYY[n] = YYY[n-1] + WWW
    end
    
#     PPP = zeros(NNN,NNN)
    PPP = Array{Float64}(undef, NNN, NNN)
    dist = Normal()
    
    for j = 1:NNN
        for k = 2:NNN-1
            PPP[j,k] = cdf(dist, (YYY[k] - λ*YYY[j] + (WWW/2))/σEps) - cdf(dist, (YYY[k] - λ*YYY[j] - (WWW/2))/σEps)        
        end
    PPP[j,1] = cdf(dist, (YYY[1] - λ*YYY[j] + (WWW/2))/σEps)
    PPP[j,NNN] = 1 - cdf(dist, (YYY[NNN] - λ*YYY[j] - (WWW/2))/σEps)
    end
    return PPP, YYY
end

In [ ]:
function ind2subv(shape, indices)
    """Map linear indices to cartesian.
    shape: d-tuple with size of each dimension.
    indices: n-iterable with linear indices.
    Returns: n-vector of d-tuples with cartesian indices.

    Based on:
    https://discourse.julialang.org/t/psa-replacement-of-ind2sub-sub2ind-in-julia-0-7/14666/8
    Similar to this matlab function:
    https://github.com/probml/pmtk3/blob/master/matlabTools/util/ind2subv.m
    """
    n = length(indices)
    d = length(shape)
    cndx = CartesianIndices(Dims(shape))
    out = Array{Tuple}(undef, n)
    for i=1:n
        lndx = indices[i]
        out[i] = cndx[lndx]
    end
    return out
end

In [ ]:
function hpfilter(y, w)
    # %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    # %  Author: Ivailo Izvorski, 
    # %          Department of Economics
    # %	   Yale University.
    # %          izvorski@econ.yale.edu
    # %  This code has been used and seems to be free of error.
    # %  However, it carries no explicit or implicit guarantee.
    # %
    # %  function [s]=hpfilter(y,w)
    # %  Hondrick Prescott filter where:
    # %  w - smoothing parameter; w=1600 for quarterly data
    # %  y - the original series that has to be smoothed
    # %  s - the filtered series
    # %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 
    
#     print(size(y)[1] < size(y)[2])
#     if size(y)[1] < size(y)[2]
#        y=y'
#     end
    
    t = size(y)[1]
    a = 6*w+1
    b = -4*w
    c = w
    d = [c b a]
    d = ones(t)*d
    m = diagm(0 => d[:,3]) + diagm(1 => d[1:t-1,2]) + diagm(-1 => d[1:t-1,2])
    m = m + diagm(2 => d[1:t-2,1]) + diagm(-2 => d[1:t-2,1])

    m[1,1] = 1+w;        m[1,2] = -2*w;
    m[2,1] = -2*w;       m[2,2] = 5*w+1;
    m[t-1,t-1] = 5*w+1;  m[t-1,t] = -2*w;
    m[t,t-1] = -2*w;     m[t,t] = 1+w;

    s=inv(m)*y 
#     s=0
    return s
end 

In [ ]:
function transformation(matrix)
    l = length(matrix)
    c = length(matrix[1])
    matrixt = zeros(l, c)
    for i in 1:l
        matrixt[i,:] = matrix[i]
    end
    return matrixt
end

In [ ]:
function fun_std_med_mean(sp, simulation_matrix; nan=false)
    @unpack Nsim = sp
    
    std_vec = zeros(Nsim)
    med_vec = zeros(Nsim)
    mean_vec = zeros(Nsim)
    corr_vec = zeros(Nsim)

    if nan==false
        for i=1:Nsim
            std_vec[i] = std(simulation_matrix[i, 2:end-1])
            med_vec[i] = median(simulation_matrix[i, 2:end-1])
            mean_vec[i] = mean(simulation_matrix[i, 2:end-1])
        end
    else
        for i=1:Nsim
            std_vec[i] = std(simulation_matrix[i, 2:end-1][.!isnan.(simulation_matrix[i, 2:end-1])])
            med_vec[i] = median(simulation_matrix[i, 2:end-1][.!isnan.(simulation_matrix[i, 2:end-1])])
            mean_vec[i] = mean(simulation_matrix[i, 2:end-1][.!isnan.(simulation_matrix[i, 2:end-1])])
        end     
    end
    
    mean_std = mean(std_vec)
    mean_med = mean(med_vec)
    mean_mean = mean(mean_vec)
    
    return mean_std, mean_med, mean_mean
end

In [ ]:
function fun_corr(sp, x, y; nan=false)
    @unpack Nsim = sp
    
    corr_vec = zeros(Nsim)

    
    if nan==false
        for i=1:Nsim
            corr_vec[i] = cor(x[i, 2:end-1], y[i, 2:end-1])
        end
    else
        for i=1:Nsim
            #y[.!isnan.(x[i, 2:end-1] -> PIB en !.NaN de x
            corr_vec[i] = cor(x[i, 2:end-1][.!isnan.(x[i, 2:end-1])], y[i, 2:end-1][.!isnan.(x[i, 2:end-1])])
        end     
    end

    mean_corr = mean(corr_vec)
    
    return mean_corr
end

In [ ]:
function fun_y_full(ae, i_c)
    @unpack grid_c, grid_yc, τ, ψ, z = ae
    
    l_c = (grid_yc ./ (1+τ) ).^(1/ψ)
    y_c = grid_yc .* l_c
    y_full = y_c .+ z*grid_c[i_c]
    return y_full
end

## Probability default and debt limit

In [ ]:
function fun_prob_default(Sim_escenario, SimDefault)
    A = Sim_escenario[:, 2:end-1][:]
    #Caso 1: Escenario malo
    B = findall(x->x==0, A)
    C = SimDefault[:, 2:end-1][:][B]
    sum_default = sum(C)
    prob_c0 = 100*sum_default/size(C[:])[1]   
    
    #Caso 2: Escenario bueno
    B = findall(x->x==1, A)
    C = SimDefault[:, 2:end-1][:][B]
    sum_default = sum(C)
    prob_c1 = 100*sum_default/size(C[:])[1]    
    
    #Caso 1: Escenarios intermedios
    B = findall(x->x==2, A)
    C = SimDefault[:, 2:end-1][:][B]    
    sum_default = sum(C)
    prob_c2 = 100*sum_default/size(C[:])[1]        
    #Prob
    return prob_c0, prob_c1, prob_c2
end

In [ ]:
function get_debt_limit(ae, i_bs, i_c; indx = false)
    @unpack Ny, prob_def, grid_b = ae
    b_p99 = Any[]
    b_p95 = Any[]
    b_p90 = Any[]
    indx_99 = Any[]
    indx_95 = Any[]
    indx_90 = Any[]
    for i_y in 1:Ny
        A = prob_def[:, i_y, i_bs, i_c]
        #bp99
        B = minimum(findall(x->x<0.01, A))
        C = grid_b[B] 
        push!(b_p99, C)
        push!(indx_99, B)
        #bp95
        B = minimum(findall(x->x<0.05, A))
        C = grid_b[B] 
        push!(b_p95, C)
        push!(indx_95, B)
        #b_p90
        B = minimum(findall(x->x<0.1, A))
        C = grid_b[B] 
        push!(b_p90, C)  
        push!(indx_90, B)
    end
    
    if indx == false
        return b_p90, b_p95, b_p99
    else
        return indx_99, indx_95, indx_90
    end
end

In [ ]:
function get_debt_limit2(ae)
    @unpack prob_def, grid_b, Ni, Ny, Nbs, Nc, grid_yc, grid_c, prob_def, grid_bs = ae
    @unpack b_c, bs_c, q, g_c = ae #Policies
    
    B = zeros(Ni, Ny, Nbs, Nc)
    BP = zeros(Ni, Ny, Nbs, Nc)
    BS = zeros(Ni, Ny, Nbs, Nc)
    BSP = zeros(Ni, Ny, Nbs, Nc)
    Q = zeros(Ni, Ny, Nbs, Nc)
    G = zeros(Ni, Ny, Nbs, Nc)
    escenario = zeros(Ni, Ny, Nbs, Nc)
    Yfull = zeros(Ni, Ny, Nbs, Nc)
    YNC   = zeros(Ni, Ny, Nbs, Nc)
    YC    = zeros(Ni, Ny, Nbs, Nc)
    Prob  = zeros(Ni, Ny, Nbs, Nc)
    
    function fun_y(ae, i_y, i_c)
        @unpack grid_c, grid_yc, τ, ψ, z = ae
        l_c = (grid_yc[i_y] / (1+τ) )^(1/ψ)
        y_c = grid_yc[i_y] * l_c
        y_full = y_c + z*grid_c[i_c]  
        return y_full
    end
    
    
    list_pr = range(0.00, 0.1, length=Ni) #Límites de probabilidad de default
    for i_y in 1:Ny
        for i_c in 1:Nc   
            y_full = fun_y(ae, i_y, i_c)
            for i_bs in 1:Nbs
                A = prob_def[:, i_y, i_bs, i_c]
                for i_i in 1:Ni #Probabilidades de default
                    i_prob = findall(x->x<=list_pr[i_i], A)  #todas las probabilidades de default menores a límite
                    A_P = A[i_prob] #Todas las prob que cumplen con criterio
                    # prob, i_b = findmax(A_P) #máxima prob que cumple con criterio y posición
                    A_C = maximum(A_P)
                    i_b = minimum(findall(x->x==A_C, A))

                    #Escenario economía
                    #Escenario malo =0
                    if (grid_yc[i_y] < 1) && (grid_c[i_c] < 1) 
                        escenario[i_i, i_y, i_bs, i_c] = 0
                    #Escenario bueno =1
                    elseif (grid_yc[i_y] > 1) && (grid_c[i_c] > 1) 
                        escenario[i_i, i_y, i_bs, i_c] = 1
                    #Escenarios intermedios =2
                    else  
                        escenario[i_i, i_y, i_bs, i_c] = 2
                    end                     
                    
                    
                    B[i_i, i_y, i_bs, i_c]  = grid_b[i_b] / y_full #Deuda presente acorde a prob de default de regla 
                    BP[i_i, i_y, i_bs, i_c] = b_c[i_b, i_y, i_bs, i_c] / y_full  #Deuda futura acorde a prob de default de regla 
                    BS[i_i, i_y, i_bs, i_c] = grid_bs[i_bs] / y_full  #Activos acorde a prob de default de regla 
                    BSP[i_i, i_y, i_bs, i_c] = bs_c[i_b, i_y, i_bs, i_c] / y_full 
                    Q[i_i, i_y, i_bs, i_c]  = q[i_b, i_y, i_bs, i_c] #Precio acorde a prob de default de regla 
                    G[i_i, i_y, i_bs, i_c]  = g_c[i_b, i_y, i_bs, i_c] / y_full  #Gasto acorde a prob de default de regla 
                    Yfull[i_i, i_y, i_bs, i_c]  = y_full
                    YNC[i_i, i_y, i_bs, i_c]    = grid_yc[i_y]
                    YC[i_i, i_y, i_bs, i_c]     = grid_c[i_c]
                    Prob[i_i, i_y, i_bs, i_c]  = prob_def[i_b, i_y, i_bs, i_c]
                
                end
            end
        end
    end
            
    return B, BP, BSP, Q, G, escenario, Yfull, YNC, YC, BS, Prob
end

In [ ]:
function get_net_debt_limit(ae, i_bs, i_c)
    @unpack Ny, prob_def, grid_b, grid_bs = ae
    b_p99 = Any[]
    b_p95 = Any[]
    b_p90 = Any[]
    for i_y in 1:Ny
        A = prob_def[:, i_y, i_bs, i_c]
        #bp99
        B = minimum(findall(x->x<0.01, A))
        C = grid_b[B] + grid_bs[i_bs]
        push!(b_p99, C)
        #bp95
        B = minimum(findall(x->x<0.05, A))
        C = grid_b[B] + grid_bs[i_bs]
        push!(b_p95, C)
        #b_p90
        B = minimum(findall(x->x<0.1, A))
        C = grid_b[B] + grid_bs[i_bs]
        push!(b_p90, C)    
    end
    
    return b_p90, b_p95, b_p99
end

## Momentos de largo plazo para reglas

In [ ]:
function momentos_reglas(ae; i_y=1, i_c=1, i_bs=1, n=500, Tsim = 1000, Nsim = 1000, 
                             mean_yx = false, mean_y=false, mean_x=false)

 """ Cobre intermedio """
    function fun_y(ae, i_y, i_c)
        @unpack grid_c, grid_yc, τ, ψ, z = ae
        l_c = (grid_yc[i_y] / (1+τ) )^(1/ψ)
        y_c = grid_yc[i_y] * l_c
        y_full = y_c + z*grid_c[i_c]  
        return y_full
    end
    
    f_round(x) = round(x, digits=5)
    
    #Nivel bajo de fondos soberanos
    y_full = fun_y(ae, i_y, i_c)
    b_p90, b_p95, b_p99 = get_debt_limit(ae, i_bs, i_c) #
    
    mean_b99 = f_round(mean(b_p99./y_full))
   
    if mean_yx == false
        #Simulación
        sp = simulation_spaces(ae,Tsim=Tsim, Nsim=Nsim)
             SimOutp, SimIntRate, SimGovt, SimDefaultEvent, SimDefault, SimDbtRatio, SimGRatio,
             SimDebt, SimB,SimBs, default_status, SimValue, SimPrice, SimBsRatio, SimFullOutp, 
             SimCopProd, SimBNRatio, Sim_escenario, SimΘ, SimΘ1, SimΘ2, SimΘ3, SimΘ4, SimBsRatiomean, 
             SimBs_escenario, SimBs_escenario_mix, SimProb = simulations(ae, sp);
    else
         #Simulación
        println("ACA")
        println((mean_yx , mean_y, mean_x))
        sp = simulation_spaces(ae,Tsim=Tsim, Nsim=Nsim)
             SimOutp, SimIntRate, SimGovt, SimDefaultEvent, SimDefault, SimDbtRatio, SimGRatio,
             SimDebt, SimB,SimBs, default_status, SimValue, SimPrice, SimBsRatio, SimFullOutp, 
             SimCopProd, SimBNRatio, Sim_escenario, SimΘ, SimΘ1, SimΘ2, SimΘ3, SimΘ4, SimBsRatiomean, 
             SimBs_escenario, SimBs_escenario_mix, SimProb = simulations(ae, sp, mean_yx = mean_yx, mean_y=mean_y, mean_x=mean_x);
       
    end
    #Funciones auxuliares
    mean_nan_x(x) = mean(x[.!isnan.(x)])  
    
    #Regla implícita
    Θ_mean  = round(mean_nan_x(SimΘ1[:, 2:end-1]),digits=4)
    
    #Ingresos de largo plazo
    y_mean = round(mean_nan_x(SimOutp[:, 2:end-1]),digits=5)
    x_mean = round(mean_nan_x(SimCopProd[:, 2:end-1]),digits=5)
    
    #Fondos soberanos de largo plazo
    Bs_mean = round(mean_nan_x(SimBs[:, 2:end-1]),digits=4)
   
    return mean_b99, Θ_mean, y_mean, x_mean, Bs_mean
end

## Principales momentos

In [ ]:
function fun_momentos(ae; n=500, Tsim = 1000, Nsim = 1000)

    #Simulación
    sp = simulation_spaces(ae,Tsim=Tsim, Nsim=Nsim)
         SimOutp, SimIntRate, SimGovt, SimDefaultEvent, SimDefault, SimDbtRatio, SimGRatio,
         SimDebt, SimB,SimBs, default_status, SimValue, SimPrice, SimBsRatio, SimFullOutp, 
         SimCopProd, SimBNRatio, Sim_escenario, SimΘ, SimΘ1, SimΘ2, SimΘ3, SimΘ4, SimBsRatiomean, 
         SimBs_escenario, SimBs_escenario_mix, SimProb = simulations(ae, sp);
    
    #Funciones auxuliares
    r3(x) = round(x, digits=3)
    mean_nan_x(x) = mean(x[.!isnan.(x)])
    med_nan_x(x) = median(x[.!isnan.(x)])
    std_nan_x(x) = std(x[.!isnan.(x)])  
    max_nan_x(x) = maximum(x[.!isnan.(x)])
    min_nan_x(x) = minimum(x[.!isnan.(x)])
    
    #Regla implícita
    mean_Θ = round(mean_nan_x(SimΘ1[:,2:end-1]),digits=4)
    std_Θ  = round(std_nan_x(SimΘ1[:,2:end-1]),digits=4)      
    
    #Momentos
    BP_Y = SimDbtRatio[:,2:end-1][:] 
    BSP_Y = SimBsRatio[:,2:end-1][:] 
    BN_Y  = SimBNRatio[:,2:end-1][:] 
    
    r_pos = findall(x-> x>0.0, SimIntRate[:,2:end-1])
    r = SimIntRate[:,2:end-1][r_pos]     
    
    gy_std, gy_med, gy_mean = fun_std_med_mean(sp, SimGRatio)


    mean_bp_y = round(mean_nan_x(BP_Y),digits=3)
    std_bp_y = round(std_nan_x(BP_Y),digits=4) 
    min_bp_y = round(min_nan_x(BP_Y),digits=3)      
    mean_bsp_y = round(mean_nan_x(BSP_Y),digits=3)
    std_bsp_y = round(std_nan_x(BSP_Y),digits=4)    
    mean_bn_y = round(mean_nan_x(BN_Y),digits=3)
    std_bn_y = round(std_nan_x(BN_Y),digits=4)       
    mean_rs = round(mean_nan_x(r),digits=4)
    med_rs = round(med_nan_x(r),digits=4)
    std_rs = round(std_nan_x(r),digits=3)  
    max_rs = round(max_nan_x(r),digits=3)    
    mean_g = round(gy_mean, digits=3)  
    std_g = round(gy_std,digits=3) 
    
    corr_y_rs = round(fun_corr(sp, SimIntRate, SimFullOutp, nan=true), digits=3)
    corr_y_gy = round(fun_corr(sp, SimGRatio, SimFullOutp, nan=true), digits=3) 
    corr_r_b = round(fun_corr(sp, SimIntRate, SimDebt, nan=true), digits=3) 
    corr_r_by = round(fun_corr(sp, SimIntRate, SimDbtRatio, nan=true), digits=3) 
    corr_r_bsy = round(fun_corr(sp, SimIntRate, SimBsRatio, nan=true), digits=3) 
    corr_r_bny = round(fun_corr(sp, SimIntRate, SimBNRatio, nan=true), digits=3) 
    corr_y_g = round(fun_corr(sp, SimGovt, SimFullOutp, nan=true), digits=3) 
    corr_y_bs  = round(fun_corr(sp, SimFullOutp, SimBs, nan=true), digits=3) 
    corr_y_bsy = round(fun_corr(sp ,SimFullOutp, SimBsRatio, nan=true), digits=3) 
    corr_y_b  = round(fun_corr(sp, SimDebt, SimFullOutp,  nan=true), digits=3) 
    corr_y_by = round(fun_corr(sp, SimDbtRatio ,SimFullOutp, nan=true), digits=3)    
    

    println("Mean of B/Y:  ", mean_bp_y)
    println("Std of B/Y:   ", std_bp_y)
    println("Min of B/Y:   ", min_bp_y)
    println("Mean of BS/Y: ", mean_bsp_y)
    println("Std of BS/Y:  ", std_bsp_y)
    println("Mean of BN/Y: ", mean_bn_y)
    println("Std of BN/Y:  ", std_bn_y) 
    println("Mean Spread:  ", 10_000*(mean_rs - ae.rf))
    println("Med Spread:   ", 10_000*(med_rs - ae.rf))
    println("Max of spread:", 10_000*(max_rs - ae.rf))    
    println("Mean of r:    ", mean_rs)
    println("Std of r:     ", std_rs)
    println("Mean of g:    ", mean_g)
    println("Std of g:     ", std_g)   
    println("Mean of Θ:    ", mean_Θ)
    println("Std of Θ:     ", std_Θ)    
    println("")    
    
    println("Corr(r,y):    ", corr_y_rs)
    println("Corr(g/y,y):  ", corr_y_gy)
    println("Corr(g,y):    ", corr_y_g)
    println("Corr(r,by):   ", corr_r_by)    
    println("Corr(r,bsy):  ", corr_r_bsy)  
    println("Corr(r,bny):  ", corr_r_bny)  
    println("Corr(y,bs):   ", corr_y_bs)    
    println("Corr(y,bsy):  ", corr_y_bsy)
    println("Corr(y,b):    ", corr_y_b)    
    println("Corr(y,by):   ", corr_y_by)
    

    #Probabilidad de default
    sim_def = SimDefault[:, 2:end-1]
    def_1 = sum(sim_def)
    pr_def = 100*def_1/size(sim_def[:])[1]

    println("N_def:", def_1)
    println("N_vec:", size(sim_def[:])[1])
    println("Probabilidad de default: ", round(pr_def, digits=3))
    
end

In [ ]:
function fun_tab_simulation(ae, dae1, dae2;
                            print_tab=true)


    

    mean_nan_x(x) = mean(x[.!isnan.(x)])
    nan_x(x) = x[.!isnan.(x)]
    id_nan(x) = .!isnan.(x)
    r(x) = round(x, digits=2)
    r4(x) = round(x, digits=4)
    # f_B(d) = findall(x -> x >= b_limit, d)
    
    sp = simulation_spaces(ae)
    SimOutp, SimIntRate, SimGovt, SimDefaultEvent, SimDefault, SimDbtRatio, SimGRatio,
    SimDebt, SimB, SimBs, default_status, SimValue, SimPrice, SimBsRatio, SimFullOutp, 
    SimCopProd, SimBNRatio, Sim_escenario, SimΘ, SimΘ1, SimΘ2, SimΘ3, SimΘ4, SimBsRatiomean, 
    SimBs_escenario, SimBs_escenario_mix, SimProb = simulations(ae, sp);
    r_pos = findall(x-> x>0.0, SimIntRate[:,2:end-1])
    rs = SimIntRate[:,2:end-1][r_pos]       
    
    W1_vec = Any[]
    W2_vec = Any[]
    P1_vec = Any[]
    P2_vec = Any[]
    
    N = length(dae1)

    
    for i in 1:N       

        sp = simulation_spaces(dae1[i])
        SimOutp_1, SimIntRate_1, SimGovt_1, SimDefaultEvent_1, SimDefault_1, SimDbtRatio_1, SimGRatio_1,
        SimDebt_1, SimB_1, SimBs_1, default_status_1, SimValue_1, SimPrice_1, SimBsRatio_1, SimFullOutp_1, 
        SimCopProd_1, SimBNRatio_1, Sim_escenario_1, SimΘ_1, SimΘ1_1, SimΘ2_1, SimΘ3_1, SimΘ4_1, SimBsRatiomean_1, 
        SimBs_escenario_1, SimBs_escenario_mix_1, SimProb_1 = simulations(dae1[i], sp);
        r_pos = findall(x-> x>0.0, SimIntRate_1[:,2:end-1])
        rs1 = SimIntRate_1[:,2:end-1][r_pos]            

        sp = simulation_spaces(dae2[i])
        SimOutp_2, SimIntRate_2, SimGovt_2, SimDefaultEvent_2, SimDefault_2, SimDbtRatio_2, SimGRatio_2,
        SimDebt_2, SimB_2, SimBs_2, default_status_2, SimValue_2, SimPrice_2, SimBsRatio_2, SimFullOutp_2, 
        SimCopProd_2, SimBNRatio_2, Sim_escenario_2, SimΘ_2, SimΘ1_2, SimΘ2_2, SimΘ3_2, SimΘ4_2, SimBsRatiomean_2, 
        SimBs_escenario_2, SimBs_escenario_mix_2, SimProb_2 = simulations(dae2[i], sp);
        r_pos = findall(x-> x>0.0, SimIntRate_2[:,2:end-1])
        rs2 = SimIntRate_2[:,2:end-1][r_pos]   
        
        if print_tab == true
    
            println("")
            println("debt limit: ", (dae1[i].b_limit, dae2[i].b_limit))
            println("                 Bench   DE   DEC")
            println("Value          :", (r(mean(SimValue[:,2:end-1])),  r(mean(SimValue_1[:,2:end-1])), r(mean(SimValue_2[:,2:end-1])) ))

            #"Mean b>b_limit"
            # i_Blim   = f_B(SimDbtRatio[:,2:end-1])
            # i_Blim_1 = f_B(SimDbtRatio_1[:,2:end-1])
            # i_Blim_2 = f_B(SimDbtRatio_2[:,2:end-1])
            # println("Value b_limit  :", (r(mean(SimValue[:,2:end-1][i_Blim])),  r(mean(SimValue_1[:,2:end-1][i_Blim_1])), r(mean(SimValue_2[:,2:end-1][i_Blim_2]))))
            
            
            println("Debt           :", (r(100*mean_nan_x(SimDbtRatio[:,2:end-1])),  r(100*mean_nan_x(SimDbtRatio_1[:,2:end-1])),  r(100*mean_nan_x(SimDbtRatio_2[:,2:end-1]))))
            println("Reserves       :", (r(100*mean(SimBsRatio[:,2:end-1])), r(100*mean(SimBsRatio_1[:,2:end-1])), r(100*mean(SimBsRatio_2[:,2:end-1])) ))

            rf = ae.rf
            println("Spread         :", (r(100*mean_nan_x(rs)  - rf),  r(100*mean_nan_x(rs1) - rf),  r(100*mean_nan_x(rs2) - rf)))
            println("G spending     :", (r(100*mean(SimGRatio[:,2:end-1])), r(100*mean(SimGRatio_1[:,2:end-1])), r(100*mean(SimGRatio_2[:,2:end-1])) ))


            #Benchmark
            sim_def = SimDefault[:, 2:end-1]
            def = sum(sim_def)
            pr_def = 100*def/length(sim_def[:])
            #Debt rule
            sim_def = SimDefault_1[:, 2:end-1]
            def_1 = sum(sim_def)
            pr_def1 = 100*def_1/length(sim_def[:])   
            #Scape clause
            sim_def = SimDefault_2[:, 2:end-1]
            def_2 = sum(sim_def)
            pr_def2 = 100*def_2/length(sim_def[:])   

            println("Default events :", (r4(pr_def), r4(pr_def1), r4(pr_def2)))
            println("N Default      :", (def, def_1, def_2))            
            println("Prob default   :", (r4(100*mean(SimProb[:,2:end-1])), r4(100*mean(SimProb_1[:,2:end-1])), r4(100*mean(SimProb_2[:,2:end-1]))))
        else
            
            W1 = mean(SimValue_1 ./ SimValue)
            W2 = mean(SimValue_2 ./ SimValue)       

            #Benchmark
            sim_def = SimDefault[:, 2:end-1]
            def_1 = sum(sim_def)
            pr_def = 100*def_1/length(sim_def[:])
            #Debt rule
            sim_def = SimDefault_1[:, 2:end-1]
            def_1 = sum(sim_def)
            pr_def1 = 100*def_1/length(sim_def[:])  
            #Scape clause
            sim_def = SimDefault_2[:, 2:end-1]
            def_1 = sum(sim_def)
            pr_def2 = 100*def_1/length(sim_def[:])   

            push!(W1_vec, W1) 
            push!(W2_vec, W2) 
            push!(P1_vec, pr_def1/pr_def) 
            push!(P2_vec, pr_def2/pr_def) 
            
        end   
    end
    
    if print_tab ==false
        return W1_vec, W2_vec, P1_vec, P2_vec
    end
    
end
       
        

In [ ]:
function fun_resumen(ae, ae_BE, ae_DE, ae_DU, ae_BEC, ae_DUC; Tsim = 1000, Nsim = 1000)

    @unpack rf = ae
    #Funciones auxuliares
    r3(x) = round(x, digits=3)
    mean_nan_x(x) = mean(x[.!isnan.(x)])
    std_nan_x(x) = std(x[.!isnan.(x)])    
        
    ae_list = [ae, ae_BE, ae_DE, ae_DU, ae_BEC, ae_DUC]
    BP_Y_list = []
    BSP_Y_list = Any[]    
    BN_Y_list = Any[]    
    G_Y_list = Any[]
    Spr_list = Any[]
    PrD_list = Any[]    
    
    for ae_i in ae_list
        #Simulación
        sp = simulation_spaces(ae_i,Tsim=Tsim, Nsim=Nsim)
        SimOutp, SimIntRate, SimGovt, SimDefaultEvent, SimDefault, SimDbtRatio, SimGRatio,
        SimDebt, SimB,SimBs, default_status, SimValue, SimPrice, SimBsRatio, SimFullOutp, 
        SimCopProd, SimBNRatio, Sim_escenario, SimΘ, SimΘ1, SimΘ2, SimΘ3, SimΘ4, SimBsRatiomean, 
        SimBs_escenario, SimBs_escenario_mix, SimProb = simulations(ae_i, sp);


        #Momentos
        BP_Y = SimDbtRatio[:,2:end-1][:] 
        BSP_Y = SimBsRatio[:,2:end-1][:] 
        BN_Y  = SimBNRatio[:,2:end-1][:] 
        G_Y = SimGRatio[:,2:end-1][:] 

        r_pos = findall(x-> x>0.0, SimIntRate[:,2:end-1])
        r = SimIntRate[:,2:end-1][r_pos]     

        mean_bp_y = round(mean_nan_x(BP_Y),digits=3)
        mean_bsp_y = round(mean_nan_x(BSP_Y),digits=3)
        mean_bn_y = round(mean_nan_x(BN_Y),digits=3)
        mean_rs = round(mean_nan_x(r),digits=3)
        mean_spr = round(10_000*(mean_rs - rf), digits=3)
        mean_g = round(mean_nan_x(G_Y),digits=3)

        #Probabilidad de default
        sim_def = SimDefault[:, 2:end-1]
        def_1 = sum(sim_def)
        pr_def =  round(100*def_1/size(sim_def[:])[1], digits=3)
        
        #Save results
        push!(BP_Y_list, mean_bp_y)
        push!(BSP_Y_list, mean_bsp_y)
        push!(BN_Y_list, mean_bn_y)
        push!(G_Y_list, mean_g)
        push!(Spr_list, mean_spr)
        push!(PrD_list, pr_def)        
    end
    
    
    println("                |     Sin cláusula               | Con cláusula")
    println("Reglas          Óptimo  | BE | Deuda | Dual      |   BE   |   Dual ")
    # println("Mean of B/Y:  ", (BP_Y_list[1], BP_Y_list[1], BP_Y_list[1], BP_Y_list[1], BP_Y_list[1], BP_Y_list[1]))
    println("Mean of B/Y:  ", [i for i in BP_Y_list[1:4]], "  | ", [i for i in BP_Y_list[5:6]])    
    println("Mean of BS/Y: ", [i for i in BSP_Y_list[1:4]], "     | ", [i for i in BSP_Y_list[5:6]])
    println("Mean of BN/Y: ", [i for i in BN_Y_list[1:4]], " | ", [i for i in BN_Y_list[5:6]])
    println("Mean of g:    ", [i for i in G_Y_list[1:4]], "     | ", [i for i in G_Y_list[5:6]])    
    println("Spread:       ", [i for i in Spr_list[1:4]], "     | ", [i for i in Spr_list[5:6]])
    println("P. default:   ", [i for i in PrD_list[1:4]], "     | ", [i for i in PrD_list[5:6]])
    
end

## Simulación escenarios

In [ ]:
function fun_sim_escenario(Sim_escenario, SimVar)
    A = Sim_escenario[:, 2:end-1][:]
    #Caso 1: Escenario malo
    B = findall(x->x==0, A)
    C0 = SimVar[:, 2:end-1][:][B]
    #Caso 2: Escenario bueno
    B = findall(x->x==1, A)
    C1 = SimVar[:, 2:end-1][:][B]
    #Caso 1: Escenarios intermedios
    B = findall(x->x==2, A)
    C2 = SimVar[:, 2:end-1][:][B]    
    return C0, C1, C2
end

In [ ]:
function fun_sim_default(Sim_escenario, SimVar)
    A = Sim_escenario[:, 2:end-1][:]
    #Caso 1: NO default
    B = findall(x->x==0, A)
    C0 = SimVar[:, 2:end-1][:][B]
    #Caso 2: Escenario default
    B = findall(x->x==1, A)
    C1 = SimVar[:, 2:end-1][:][B]

    return C0, C1
end

In [ ]:
function fun_sim_findθ(SimΘ)
    A = SimΘ[:, 2:end-1][:]
    B = findall(x->x<-0.015, A)
    #Entrega la posición cuando theta está fuera de regla
    return B
end

In [ ]:
function fun_sim_theta_escenarios(ae; Tsim=1000, Nsim=1000)
    #Simulación
    sp = simulation_spaces(ae,Tsim=Tsim, Nsim=Nsim)
    SimOutp, SimIntRate, SimGovt, SimDefaultEvent, SimDefault, SimDbtRatio, SimGRatio,
    SimDebt, SimB,SimBs, default_status, SimValue, SimPrice, SimBsRatio, SimFullOutp, 
    SimCopProd, SimBNRatio, Sim_escenario, SimΘ, SimΘ1, SimΘ2, SimΘ3, SimΘ4, SimBsRatiomean, 
    SimBs_escenario, SimBs_escenario_mix, SimProb = simulations(ae, sp);
    
    Θ1, Θ2, Θ3  = fun_sim_escenario(Sim_escenario, SimΘ1)

    r(x) = round(x, digits=3)
    mean_nan_x(x) = mean(x[.!isnan.(x)])
    mean_Θ1 = r(mean_nan_x(Θ1))
    mean_Θ2 = r(mean_nan_x(Θ2))
    mean_Θ3 = r(mean_nan_x(Θ3))
    
    println("Escenarios de Θ según ciclo económico")
    println("Escenario adverso:    ", mean_Θ1)
    println("Escenario intermedio: ", mean_Θ3)
    println("Escenario bueno:      ", mean_Θ2)
    
end

In [ ]:
function fun_sim_escenario_mix(Sim_escenario, SimVar; table=true, tcount=false)
    A = Sim_escenario[:, 2:end-1][:]
    
    """ Deuda alta """
    #Caso 0: Deuda alta y ciclo adverso
    B = findall(x->x==0, A)
    C0 = SimVar[:, 2:end-1][:][B]
    #Caso 1: Deuda alta y bueno
    B = findall(x->x==1, A)
    C1 = SimVar[:, 2:end-1][:][B]
    #Caso 2: Deuda alta y ciclo intermedio
    B = findall(x->x==2, A)
    C2 = SimVar[:, 2:end-1][:][B]    
    
    """ Deuda baja """
    #Caso 3: Deuda baja y ciclo adverso
    B = findall(x->x==3, A)
    C3 = SimVar[:, 2:end-1][:][B]
    #Caso 4: Deuda baja y bueno
    B = findall(x->x==4, A)
    C4 = SimVar[:, 2:end-1][:][B]
    #Caso 5: Deuda baja y ciclo intermedio
    B = findall(x->x==5, A)
    C5 = SimVar[:, 2:end-1][:][B]        
    
    """ Deuda intermedia """
    #Caso 6: Deuda intermedia y ciclo adverso
    B = findall(x->x==6, A)
    C6 = SimVar[:, 2:end-1][:][B]
    #Caso 7: Deuda intermedia y bueno
    B = findall(x->x==7, A)
    C7 = SimVar[:, 2:end-1][:][B]
    #Caso 8: Deuda intermedia y ciclo intermedio
    B = findall(x->x==8, A)
    C8 = SimVar[:, 2:end-1][:][B]    
    
    r(x) = round(x, digits=3)
    mean_nan_x(x) = mean(x[.!isnan.(x)])    
    
    if table == true
        if tcount == false
            C0 = r(mean_nan_x(C0))
            C1 = r(mean_nan_x(C1))
            C2 = r(mean_nan_x(C2))
            C3 = r(mean_nan_x(C3))
            C4 = r(mean_nan_x(C4))
            C5 = r(mean_nan_x(C5))
            C6 = r(mean_nan_x(C6))
            C7 = r(mean_nan_x(C7))
            C8 = r(mean_nan_x(C8))
        else
            
            C0 = length(C0)
            C1 = length(C1)
            C2 = length(C2)
            C3 = length(C3)
            C4 = length(C4)
            C5 = length(C5)
            C6 = length(C6)
            C7 = length(C7)
            C8 = length(C8)
        end
        # println("Fondos soberanos")
        println("Deuda bruta /        Ciclo económico")
        println("                Adverso Intermedio Bueno")
        println("Baja:            ", (C3, C5, C4))
        println("Intermedia:      ", (C6, C8, C7))
        println("Alta:            ", (C0, C2, C1))
    else
        return C0, C1, C2, C3, C4, C5, C6, C8, C8 
    end
end

In [ ]:
function fun_sim_fs_escenarios_mix(ae; ratio_mean = false, table=true, Tsim=1000, Nsim=1000)
    sp = simulation_spaces(ae,Tsim=Tsim, Nsim=Nsim)
    SimOutp, SimIntRate, SimGovt, SimDefaultEvent, SimDefault, SimDbtRatio, SimGRatio,
    SimDebt, SimB,SimBs, default_status, SimValue, SimPrice, SimBsRatio, SimFullOutp, 
    SimCopProd, SimBNRatio, Sim_escenario, SimΘ, SimΘ1, SimΘ2, SimΘ3, SimΘ4, SimBsRatiomean, 
    SimBs_escenario, SimBs_escenario_mix, SimProb = simulations(ae, sp);  
    
    if ratio_mean == false
        println("Fondos soberanos sobre PIB según deuda bruta y ciclo económico")
        fun_sim_escenario_mix(SimBs_escenario_mix, SimBsRatio; table=table)
        println("")
        println("Spread según deuda bruta y ciclo económico")
        spread = (SimIntRate .- ae.rf).*10_000
        fun_sim_escenario_mix(SimBs_escenario_mix, spread; table=table)   
        println("")
        println("Deuda bruta sobre PIB según deuda bruta y ciclo económico")
        fun_sim_escenario_mix(SimBs_escenario_mix, SimDbtRatio; table=table)        
        println("")
        println("Número de observaciones por escenario")
        fun_sim_escenario_mix(SimBs_escenario_mix, SimBsRatio; table=table, tcount=true)           
    else
        println("Fondos soberanos sobre PIB promedio según deuda bruta y ciclo económico")
        fun_sim_escenario_mix(SimBs_escenario_mix, SimBsRatiomean; table=table)
        println("")
        println("Spread según según deuda bruta y ciclo económico")
        spread = (SimIntRate .- ae.rf).*10_000
        fun_sim_escenario_mix(SimBs_escenario_mix, spread; table=table)   
        println("")
        println("Deuda bruta sobre PIB según deuda bruta y ciclo económico")
        fun_sim_escenario_mix(SimBs_escenario_mix, SimDbtRatio; table=table) 
        println("")
        println("Número de observaciones por escenario")
        fun_sim_escenario_mix(SimBs_escenario_mix, SimBsRatio; table=table, tcount=true)                    
    end
end

In [ ]:
function table_debt_g_escenario(ae; Tsim = 1000, Nsim = 1000, 
                                rmean="mean")
    
    #Simulación modelo óptimo
    sp = simulation_spaces(ae, Tsim=Tsim, Nsim=Nsim)
    SimOutp, SimIntRate, SimGovt, SimDefaultEvent, SimDefault, SimDbtRatio, SimGRatio,
    SimDebt, SimB,SimBs, default_status, SimValue, SimPrice, SimBsRatio, SimFullOutp, 
    SimCopProd, SimBNRatio, Sim_escenario, SimΘ, SimΘ1, SimΘ2, SimΘ3, SimΘ4, SimBsRatiomean, 
    SimBs_escenario, SimBs_escenario_mix, SimProb = simulations(ae, sp);    
    
    
    function mean_nan_x(x)
        if rmean == "mean"
            return  mean(x[.!isnan.(x)])
        elseif rmean == "median"
            return  median(x[.!isnan.(x)])
        elseif rmean == "std"
            return  std(x[.!isnan.(x)])
        end
    end
        
        
    r(x) = round(x, digits=3)
    #Gasto
    GY1, GY2, GY3 = fun_sim_escenario(Sim_escenario, SimGRatio)
    m_gy_b = r(mean_nan_x(GY2))
    m_gy_i = r(mean_nan_x(GY3))
    m_gy_a = r(mean_nan_x(GY1))
    
    G1, G2, G3 = fun_sim_escenario(Sim_escenario, SimGovt)
    m_g_b = r(mean_nan_x(G2))
    m_g_i = r(mean_nan_x(G3))
    m_g_a = r(mean_nan_x(G1))    
    
    Y1, Y2, Y3 = fun_sim_escenario(Sim_escenario, SimOutp)
    m_y_b = r(mean_nan_x(Y2))
    m_y_i = r(mean_nan_x(Y3))
    m_y_a = r(mean_nan_x(Y1))    
        
    #Deuda bruta
    B1, B2, B3 = fun_sim_escenario(Sim_escenario, SimDbtRatio)
    m_B_b = r(mean_nan_x(B2))
    m_B_i = r(mean_nan_x(B3))
    m_B_a = r(mean_nan_x(B1))
    
   #Fondos soberanos
    BS1, BS2, BS3 = fun_sim_escenario(Sim_escenario, SimBsRatio)
    m_BS_b = r(mean_nan_x(BS2))
    m_BS_i = r(mean_nan_x(BS3))
    m_BS_a = r(mean_nan_x(BS1))  
    
    #Deuda neta
    BN1, BN2, BN3 = fun_sim_escenario(Sim_escenario, SimBNRatio)   
    m_BN_b = r(mean_nan_x(BN2))
    m_BN_i = r(mean_nan_x(BN3))
    m_BN_a = r(mean_nan_x(BN1)) 
    
    #Spread
    r1, r2, r3 = fun_sim_escenario(Sim_escenario, SimIntRate)  
    # mean_rs = round(mean_nan_x(r),digits=4)
    # println("Mean Spread:  ", 10_000*(mean_rs - ae.rf))

    rf = ae.rf
    m_r_b = r((mean_nan_x(r2) - rf)*10_000) 
    m_r_i = r((mean_nan_x(r3) - rf)*10_000) 
    m_r_a = r((mean_nan_x(r1) - rf)*10_000)     
    
    
    #Probability default
    Pr1, Pr2, Pr3 = fun_sim_escenario(Sim_escenario, SimDefault)  
    
    def_1 = sum(Pr1)
    pr_def1 = r(100*def_1/size(Pr1[:])[1] )   
    
    def_2 = sum(Pr2)
    pr_def2 = r(100*def_2/size(Pr2[:])[1] )  
    
    def_3 = sum(Pr3)
    pr_def3 = r(100*def_3/size(Pr3[:])[1] )  
    

    
    println("Escenario         Bueno  Intermedio  Adverso")
    println("Gasto fiscal/GDP: " , (m_gy_b, m_gy_i, m_gy_a))
    println("Gasto fiscal:     " , (m_g_b, m_g_i, m_g_a))
    println("GDP no cobre:     " , (m_y_b, m_y_i, m_y_a))
    println("Deuda bruta:      " , (m_B_b, m_B_i, m_B_a))
    println("Fondos soberanos: " , (m_BS_b, m_BS_i, m_BS_a))
    println("Deuda neta:       " , (m_BN_b, m_BN_i, m_BN_a))
    println("Spread:           " , (m_r_b, m_r_i, m_r_a))    
    println("Prob. Default(%): " , (pr_def2, pr_def3, pr_def1))    
end

In [2]:
function table_debt_g_escenario_blimit(ae; Tsim = 1000, Nsim = 1000, 
                                rmean="mean", blimit=-0.2)
    
    #TABLA CUANDO LA DEUDA ES MAYOR A CIERTO NIVEL
    #Simulación modelo óptimo
    sp = simulation_spaces(ae, Tsim=Tsim, Nsim=Nsim)
    SimOutp, SimIntRate, SimGovt, SimDefaultEvent, SimDefault, SimDbtRatio, SimGRatio,
    SimDebt, SimB,SimBs, default_status, SimValue, SimPrice, SimBsRatio, SimFullOutp, 
    SimCopProd, SimBNRatio, Sim_escenario, SimΘ, SimΘ1, SimΘ2, SimΘ3, SimΘ4, SimBsRatiomean, 
    SimBs_escenario, SimBs_escenario_mix, SimProb = simulations(ae, sp);    
    
    
    function mean_nan_x(x)
        if rmean == "mean"
            return  mean(x[.!isnan.(x)])
        elseif rmean == "median"
            return  median(x[.!isnan.(x)])
        elseif rmean == "std"
            return  std(x[.!isnan.(x)])
        end
    end
        
    not_nan(x) = round.(x[.!isnan.(x)], digits=3)
        
    r(x) = round(x, digits=4)
    
    #Deuda bruta
    B1, B2, B3 = fun_sim_escenario(Sim_escenario, SimDbtRatio)
    i_B1 = findall(x -> x < blimit, B1) #Niveles de deuda más alto
    i_B2 = findall(x -> x < blimit, B2) #Niveles de deuda más alto
    i_B3 = findall(x -> x < blimit, B3) #Niveles de deuda más alto
    m_B_b = r(mean_nan_x(B2[i_B2]))
    m_B_i = r(mean_nan_x(B3[i_B3]))
    m_B_a = r(mean_nan_x(B1[i_B1]))
    
    i_B4 = findall(x -> x >= blimit, B1) #Niveles de deuda más alto
    i_B5 = findall(x -> x >= blimit, B2) #Niveles de deuda más alto
    i_B6 = findall(x -> x >= blimit, B3) #Niveles de deuda más alto    
    
    # println((size(Sim_escenario), size(Sim_escenario[:,2:end-1])))
    
    xx1 = size(B2[i_B2])[1] + size(B3[i_B3])[1] + size(B1[i_B1])[1]
    xx2 = size(B2[i_B5])[1] + size(B3[i_B6])[1] + size(B1[i_B4])[1]
    println("                total        B         I         A")
    println("B<blimit: ",(size(Sim_escenario), size(B2[i_B2]), size(B3[i_B3]), size(B1[i_B1])))
    println("B>=blimit:",(size(Sim_escenario), size(B2[i_B5]), size(B3[i_B6]), size(B1[i_B4])))
    println("Prop>",-100*blimit, " : ", round(100*xx1/(xx1+xx2), digits=1))
    
    
    # println((size(B1[i_B1]), size(i_B1)))
    #Gasto
    GY1, GY2, GY3 = fun_sim_escenario(Sim_escenario, SimGRatio)
    
    m_gy_b = r(mean_nan_x(GY2[i_B2]))
    m_gy_i = r(mean_nan_x(GY3[i_B3]))
    m_gy_a = r(mean_nan_x(GY1[i_B1]))
    
    G1, G2, G3 = fun_sim_escenario(Sim_escenario, SimGovt)
    m_g_b = r(mean_nan_x(G2[i_B2]))
    m_g_i = r(mean_nan_x(G3[i_B3]))
    m_g_a = r(mean_nan_x(G1[i_B1]))    
    
    Y1, Y2, Y3 = fun_sim_escenario(Sim_escenario, SimOutp)
    m_y_b = r(mean_nan_x(Y2[i_B2]))
    m_y_i = r(mean_nan_x(Y3[i_B3]))
    m_y_a = r(mean_nan_x(Y1[i_B1]))    
      
   #Fondos soberanos
    BS1, BS2, BS3 = fun_sim_escenario(Sim_escenario, SimBsRatio)
    m_BS_b = r(mean_nan_x(BS2[i_B2]))
    m_BS_i = r(mean_nan_x(BS3[i_B3]))
    m_BS_a = r(mean_nan_x(BS1[i_B1]))  
    
    #Deuda neta
    BN1, BN2, BN3 = fun_sim_escenario(Sim_escenario, SimBNRatio)   
    m_BN_b = r(mean_nan_x(BN2[i_B2]))
    m_BN_i = r(mean_nan_x(BN3[i_B3]))
    m_BN_a = r(mean_nan_x(BN1[i_B1])) 
    
    
    #Spread
    r1, r2, r3 = fun_sim_escenario(Sim_escenario, SimIntRate)  
    # mean_rs = round(mean_nan_x(r),digits=4)
    # println("Mean Spread:  ", 10_000*(mean_rs - ae.rf))

    # println("       A     I     B")
    # println("Max:", (maximum(not_nan(r1)), maximum(not_nan(r3)), maximum(not_nan(r2))))
    # println("Min:", (minimum(not_nan(r1)), minimum(not_nan(r3)), minimum(not_nan(r2))))
    # pl1 = histogram(r1, color="red", label="A", bins=100, xlims=(-0.01,0.1), alpha=0.7)
    # pl1 = histogram(r2, color="blue", label="B", bins=100, alpha=0.7)
    # pl1 = histogram!(r3, color="green", label="I", bins=100, alpha=0.7)
    # display(pl1)
    
    
    rf = ae.rf
    m_r_b = r((mean_nan_x(r2[i_B2]) - rf)*10_000) 
    m_r_i = r((mean_nan_x(r3[i_B3]) - rf)*10_000) 
    m_r_a = r((mean_nan_x(r1[i_B1]) - rf)*10_000)     
    
    #Probability default
    Pr1, Pr2, Pr3 = fun_sim_escenario(Sim_escenario, SimDefault)  

    Pr1 = Pr1[i_B1]
    Pr2 = Pr2[i_B2]
    Pr3 = Pr3[i_B3]

    def_1 = sum(Pr1)
    pr_def1 = r(100*def_1/size(Pr1[:])[1] )   
    
    def_2 = sum(Pr2)
    pr_def2 = r(100*def_2/size(Pr2[:])[1] )  
    
    def_3 = sum(Pr3)
    pr_def3 = r(100*def_3/size(Pr3[:])[1] )  

    
    println("Escenario         Bueno  Intermedio  Adverso")
    println("Gasto fiscal/GDP: " , (m_gy_b, m_gy_i, m_gy_a))
    println("Gasto fiscal:     " , (m_g_b, m_g_i, m_g_a))
    println("GDP no cobre:     " , (m_y_b, m_y_i, m_y_a))
    println("Deuda bruta:      " , (m_B_b, m_B_i, m_B_a))
    println("Fondos soberanos: " , (m_BS_b, m_BS_i, m_BS_a))
    println("Deuda neta:       " , (m_BN_b, m_BN_i, m_BN_a))
    println("Spread:           " , (m_r_b, m_r_i, m_r_a))    
    println("Prob. Default(%): " , (pr_def2, pr_def3, pr_def1))    
end

table_debt_g_escenario_blimit (generic function with 1 method)

## Tabla deuda prudente

In [ ]:
function table__deuda_bruta_prudente(ae)
    @unpack Nc, Nbs = ae
    
    f_round(x) = round(x, digits=3)
    #índices niveles de fondos soberanos
    i_bs_i = Int(floor(Nbs/2) + 1) # Bs intermedio
    i_bs_h, i_bs_l = Int(floor(Nbs*0.8)), Int(floor(Nbs*0.2))

    """ Cobre bajo """
    i_c = Int(floor(Nc*0.2))
    #Nivel bajo de fondos soberanos
    y_full = fun_y_full(ae, i_c)
    b_p90, b_p95, b_p99 = get_debt_limit(ae, i_bs_l, i_c)

    mean_b90_bsl_cl = f_round(mean(b_p90./y_full))
    mean_b95_bsl_cl = f_round(mean(b_p95./y_full))
    mean_b99_bsl_cl = f_round(mean(b_p99./y_full))

    #Nivel intermedio de fondos soberanos
    y_full = fun_y_full(ae, i_c)
    b_p90, b_p95, b_p99 = get_debt_limit(ae, i_bs_i, i_c)

    mean_b90_bsi_cl = f_round(mean(b_p90./y_full))
    mean_b95_bsi_cl = f_round(mean(b_p95./y_full))
    mean_b99_bsi_cl = f_round(mean(b_p99./y_full))
    
    #Nivel alto de fondos soberanos
    y_full = fun_y_full(ae, i_c)
    b_p90, b_p95, b_p99 = get_debt_limit(ae, i_bs_h, i_c)

    mean_b90_bsh_cl = f_round(mean(b_p90./y_full))
    mean_b95_bsh_cl = f_round(mean(b_p95./y_full))
    mean_b99_bsh_cl = f_round(mean(b_p99./y_full))      
    
    """ Cobre intermedio """
    i_c = Int(floor(Nc/2) + 1)
    #Nivel bajo de fondos soberanos
    y_full = fun_y_full(ae, i_c)
    b_p90, b_p95, b_p99 = get_debt_limit(ae, i_bs_l, i_c)

    mean_b90_bsl_ci = f_round(mean(b_p90./y_full))
    mean_b95_bsl_ci = f_round(mean(b_p95./y_full))
    mean_b99_bsl_ci = f_round(mean(b_p99./y_full))

    #Nivel intermedio de fondos soberanos
    y_full = fun_y_full(ae, i_c)
    b_p90, b_p95, b_p99 = get_debt_limit(ae, i_bs_i, i_c)

    mean_b90_bsi_ci = f_round(mean(b_p90./y_full))
    mean_b95_bsi_ci = f_round(mean(b_p95./y_full))
    mean_b99_bsi_ci = f_round(mean(b_p99./y_full))
    
    #Nivel alto de fondos soberanos
    y_full = fun_y_full(ae, i_c)
    b_p90, b_p95, b_p99 = get_debt_limit(ae, i_bs_h, i_c)

    mean_b90_bsh_ci = f_round(mean(b_p90./y_full))
    mean_b95_bsh_ci = f_round(mean(b_p95./y_full))
    mean_b99_bsh_ci = f_round(mean(b_p99./y_full))      
        
    
    """ Cobre alto """
    i_c = Int(floor(Nc*0.8))    
    #Nivel bajo de fondos soberanos
    y_full = fun_y_full(ae, i_c)
    b_p90, b_p95, b_p99 = get_debt_limit(ae, i_bs_l, i_c)

    mean_b90_bsl_ch = f_round(mean(b_p90./y_full))
    mean_b95_bsl_ch = f_round(mean(b_p95./y_full))
    mean_b99_bsl_ch = f_round(mean(b_p99./y_full))

    #Nivel intermedio de fondos soberanos
    y_full = fun_y_full(ae, i_c)
    b_p90, b_p95, b_p99 = get_debt_limit(ae, i_bs_i, i_c)

    mean_b90_bsi_ch = f_round(mean(b_p90./y_full))
    mean_b95_bsi_ch = f_round(mean(b_p95./y_full))
    mean_b99_bsi_ch = f_round(mean(b_p99./y_full))
    
    #Nivel alto de fondos soberanos
    y_full = fun_y_full(ae, i_c)
    b_p90, b_p95, b_p99 = get_debt_limit(ae, i_bs_h, i_c)

    mean_b90_bsh_ch = f_round(mean(b_p90./y_full))
    mean_b95_bsh_ch = f_round(mean(b_p95./y_full))
    mean_b99_bsh_ch = f_round(mean(b_p99./y_full))         
    
    println("Deuda bruta prudente ")
    println("                      Fondos soberanos")
    println("Cobre bajo/        Bajo  Intermedio  Alto")
    println("Prob 10%:        ", (mean_b90_bsl_cl, mean_b90_bsi_cl, mean_b90_bsh_cl))
    println("Prob 5% :        ", (mean_b95_bsl_cl, mean_b95_bsi_cl, mean_b95_bsh_cl))
    println("Prob 1% :        ", (mean_b99_bsl_cl, mean_b99_bsi_cl, mean_b99_bsh_cl))
    
    println("")
    println("Cobre intermedio/  Bajo  Intermedio  Alto")
    println("Prob 10%:        ", (mean_b90_bsl_ci, mean_b90_bsi_ci, mean_b90_bsh_ci))
    println("Prob 5% :        ", (mean_b95_bsl_ci, mean_b95_bsi_ci, mean_b95_bsh_ci))
    println("Prob 1% :        ", (mean_b99_bsl_ci, mean_b99_bsi_ci, mean_b99_bsh_ci))    
    
    println("")
    println("Cobre alto/        Bajo  Intermedio  Alto")
    println("Prob 10%:        ", (mean_b90_bsl_ch, mean_b90_bsi_ch, mean_b90_bsh_ch))
    println("Prob 5% :        ", (mean_b95_bsl_ch, mean_b95_bsi_ch, mean_b95_bsh_ch))
    println("Prob 1% :        ", (mean_b99_bsl_ch, mean_b99_bsi_ch, mean_b99_bsh_ch))    
    
end
    

In [ ]:
function table__deuda_neta_prudente(ae)
    @unpack Nc, Nbs = ae
    
    f_round(x) = round(x, digits=3)
    #índices niveles de fondos soberanos
    i_bs_i = Int(floor(Nbs/2) + 1) # Bs intermedio
    i_bs_h, i_bs_l = Int(floor(Nbs*0.8)), Int(floor(Nbs*0.2))

    """ Cobre bajo """
    i_c = Int(floor(Nc*0.2))
    #Nivel bajo de fondos soberanos
    y_full = fun_y_full(ae, i_c)
    b_p90, b_p95, b_p99 = get_net_debt_limit(ae, i_bs_l, i_c) #
    
    mean_b90_bsl_cl = f_round(mean(b_p90./y_full))
    mean_b95_bsl_cl = f_round(mean(b_p95./y_full))
    mean_b99_bsl_cl = f_round(mean(b_p99./y_full))

    #Nivel intermedio de fondos soberanos
    y_full = fun_y_full(ae, i_c)
    b_p90, b_p95, b_p99 = get_net_debt_limit(ae, i_bs_i, i_c)

    mean_b90_bsi_cl = f_round(mean(b_p90./y_full))
    mean_b95_bsi_cl = f_round(mean(b_p95./y_full))
    mean_b99_bsi_cl = f_round(mean(b_p99./y_full))
    
    #Nivel alto de fondos soberanos
    y_full = fun_y_full(ae, i_c)
    b_p90, b_p95, b_p99 = get_net_debt_limit(ae, i_bs_h, i_c)

    mean_b90_bsh_cl = f_round(mean(b_p90./y_full))
    mean_b95_bsh_cl = f_round(mean(b_p95./y_full))
    mean_b99_bsh_cl = f_round(mean(b_p99./y_full))      
    
    """ Cobre intermedio """
    i_c = Int(floor(Nc/2) + 1)
    #Nivel bajo de fondos soberanos
    y_full = fun_y_full(ae, i_c)
    b_p90, b_p95, b_p99 = get_net_debt_limit(ae, i_bs_l, i_c)

    mean_b90_bsl_ci = f_round(mean(b_p90./y_full))
    mean_b95_bsl_ci = f_round(mean(b_p95./y_full))
    mean_b99_bsl_ci = f_round(mean(b_p99./y_full))

    #Nivel intermedio de fondos soberanos
    y_full = fun_y_full(ae, i_c)
    b_p90, b_p95, b_p99 = get_net_debt_limit(ae, i_bs_i, i_c)

    mean_b90_bsi_ci = f_round(mean(b_p90./y_full))
    mean_b95_bsi_ci = f_round(mean(b_p95./y_full))
    mean_b99_bsi_ci = f_round(mean(b_p99./y_full))
    
    #Nivel alto de fondos soberanos
    y_full = fun_y_full(ae, i_c)
    b_p90, b_p95, b_p99 = get_net_debt_limit(ae, i_bs_h, i_c)

    mean_b90_bsh_ci = f_round(mean(b_p90./y_full))
    mean_b95_bsh_ci = f_round(mean(b_p95./y_full))
    mean_b99_bsh_ci = f_round(mean(b_p99./y_full))      
        
    
    """ Cobre alto """
    i_c = Int(floor(Nc*0.8))    
    #Nivel bajo de fondos soberanos
    y_full = fun_y_full(ae, i_c)
    b_p90, b_p95, b_p99 = get_net_debt_limit(ae, i_bs_l, i_c)

    mean_b90_bsl_ch = f_round(mean(b_p90./y_full))
    mean_b95_bsl_ch = f_round(mean(b_p95./y_full))
    mean_b99_bsl_ch = f_round(mean(b_p99./y_full))

    #Nivel intermedio de fondos soberanos
    y_full = fun_y_full(ae, i_c)
    b_p90, b_p95, b_p99 = get_net_debt_limit(ae, i_bs_i, i_c)

    mean_b90_bsi_ch = f_round(mean(b_p90./y_full))
    mean_b95_bsi_ch = f_round(mean(b_p95./y_full))
    mean_b99_bsi_ch = f_round(mean(b_p99./y_full))
    
    #Nivel alto de fondos soberanos
    y_full = fun_y_full(ae, i_c)
    b_p90, b_p95, b_p99 = get_net_debt_limit(ae, i_bs_h, i_c)

    mean_b90_bsh_ch = f_round(mean(b_p90./y_full))
    mean_b95_bsh_ch = f_round(mean(b_p95./y_full))
    mean_b99_bsh_ch = f_round(mean(b_p99./y_full))         
    
    println("Deuda neta prudente ")
    println("                      Fondos soberanos")
    println("Cobre bajo/        Bajo  Intermedio  Alto")
    println("Prob 10%:        ", (mean_b90_bsl_cl, mean_b90_bsi_cl, mean_b90_bsh_cl))
    println("Prob 5% :        ", (mean_b95_bsl_cl, mean_b95_bsi_cl, mean_b95_bsh_cl))
    println("Prob 1% :        ", (mean_b99_bsl_cl, mean_b99_bsi_cl, mean_b99_bsh_cl))
    
    println("")
    println("Cobre intermedio/  Bajo  Intermedio  Alto")
    println("Prob 10%:        ", (mean_b90_bsl_ci, mean_b90_bsi_ci, mean_b90_bsh_ci))
    println("Prob 5% :        ", (mean_b95_bsl_ci, mean_b95_bsi_ci, mean_b95_bsh_ci))
    println("Prob 1% :        ", (mean_b99_bsl_ci, mean_b99_bsi_ci, mean_b99_bsh_ci))    
    
    println("")
    println("Cobre alto/        Bajo  Intermedio  Alto")
    println("Prob 10%:        ", (mean_b90_bsl_ch, mean_b90_bsi_ch, mean_b90_bsh_ch))
    println("Prob 5% :        ", (mean_b95_bsl_ch, mean_b95_bsi_ch, mean_b95_bsh_ch))
    println("Prob 1% :        ", (mean_b99_bsl_ch, mean_b99_bsi_ch, mean_b99_bsh_ch))    
    
end
    

In [ ]:
function table__deuda_bruta_catastrofe(ae)
    @unpack Nc, Nbs, Ny = ae
    
    #Funciones auxuliares
    function fun_y(ae, i_y, i_c)
        @unpack grid_c, grid_yc, τ, ψ, z = ae
        l_c = (grid_yc[i_y] / (1+τ) )^(1/ψ)
        y_c = grid_yc[i_y] * l_c
        y_full = y_c + z*grid_c[i_c]  
        return y_full
    end
    
    f_round(x) = round(x, digits=3)
    
    #índices niveles de fondos soberanos
    i_bs = 1

    """ Escenario 1% """
    i_c = 1 
    i_y = 1 
    
    #Nivel bajo de fondos soberanos
    y_full = fun_y(ae, i_y, i_c)
    b_p90, b_p95, b_p99 = get_debt_limit(ae, i_bs, i_c) #
    
    mean_b90_p1 = f_round(mean(b_p90./y_full))
    mean_b95_p1 = f_round(mean(b_p95./y_full))
    mean_b99_p1 = f_round(mean(b_p99./y_full))

    """ Escenario 5% """
    i_c = Int(floor(Nc*0.05)+1)    
    i_y = Int(floor(Ny*0.05)+1)   
    
    y_full = fun_y(ae, i_y, i_c)
    b_p90, b_p95, b_p99 = get_debt_limit(ae, i_bs, i_c)

    mean_b90_p5 = f_round(mean(b_p90./y_full))
    mean_b95_p5 = f_round(mean(b_p95./y_full))
    mean_b99_p5 = f_round(mean(b_p99./y_full))
    
    """ Escenario 10% """
    i_c = Int(floor(Nc*0.1)+1)    
    i_y = Int(floor(Ny*0.1)+1)      
    y_full = fun_y(ae, i_y, i_c)
    b_p90, b_p95, b_p99 = get_debt_limit(ae, i_bs, i_c)

    mean_b90_p10 = f_round(mean(b_p90./y_full))
    mean_b95_p10 = f_round(mean(b_p95./y_full))
    mean_b99_p10 = f_round(mean(b_p99./y_full))      
    
      
    
    println("Deuda bruta prudente ")
    println("                Escenario cobre y PIB")
    println("                     1%    10%")
    println("Prob 10%:        ", (mean_b90_p1, mean_b90_p10))
    println("Prob 5% :        ", (mean_b95_p1, mean_b95_p10))
    println("Prob 1% :        ", (mean_b99_p1, mean_b99_p10))
      
    
end

In [ ]:
function table_deuda_bruta(ae; indx_bs=false, extremo=false)
    @unpack Nc, Nbs, Ny, q, g_c, grid_bs, rf, bs_c = ae
    
    #Funciones auxuliares
    function fun_y(ae, i_y, i_c)
        @unpack grid_c, grid_yc, τ, ψ, z = ae
        l_c = (grid_yc[i_y] / (1+τ) )^(1/ψ)
        y_c = grid_yc[i_y] * l_c
        y_full = y_c + z*grid_c[i_c]  
        return y_full
    end
    
    f_round(x,d=3) = round(x, digits=d)
    
    """ índices nivel de fondos soberanos """
    if indx_bs==false
        i_bs = Int(floor(Nbs/2)+1)    
    else
        i_bs = indx_bs
    end
    
    """ Escenario bajo """
    #Cobre y pib
    if extremo == false
        i_c = Int(floor(Nc*0.1)+1) 
        i_y = Int(floor(Ny*0.1)+1)   
    else
        i_c = Int(floor(Nc*0.01)+1) 
        i_y = Int(floor(Ny*0.01)+1)           
    end
    
    #Deuda
    y_full = fun_y(ae, i_y, i_c)
    b_p90, b_p95, b_p99 = get_debt_limit(ae, i_bs, i_c) #
    
    mean_b90 = f_round(mean(b_p90./y_full))
    mean_b95 = f_round(mean(b_p95./y_full))
    mean_b99 = f_round(mean(b_p99./y_full))  
    
    #Índice deuda prudente
    indx_99, indx_95, indx_90 = get_debt_limit(ae, i_bs, i_c, indx=true)    

    #Activos
    bs_y = f_round(grid_bs[i_bs] /y_full)
    bs_y_p90 = f_round(bs_c[indx_90[i_y], i_y, i_bs, i_c]./y_full, 5) 
    bs_y_p95 = f_round(bs_c[indx_95[i_y], i_y, i_bs, i_c]./y_full, 5) 
    bs_y_p99 = f_round(bs_c[indx_99[i_y], i_y, i_bs, i_c]./y_full, 5) 
        
    
    #Tasa de interés
    r_90 =  f_round((1 ./ q[indx_90[i_y], i_y, i_bs, i_c] .-1 - rf)*10_000)
    r_95 =  f_round((1 ./ q[indx_95[i_y], i_y, i_bs, i_c] .-1 - rf)*10_000)
    r_99 =  f_round((1 ./ q[indx_99[i_y], i_y, i_bs, i_c] .-1 - rf)*10_000)

    # r_90 =  f_round((1 ./ q[indx_90[i_y], i_y, i_bs, i_c] .-1 ))
    # r_95 =  f_round((1 ./ q[indx_95[i_y], i_y, i_bs, i_c] .-1 ))
    # r_99 =  f_round((1 ./ q[indx_99[i_y], i_y, i_bs, i_c] .-1 ))
        
    
    #Gasto
    g_90 = f_round(g_c[indx_90[i_y], i_y, i_bs, i_c]./y_full)
    g_95 = f_round(g_c[indx_95[i_y], i_y, i_bs, i_c]./y_full)
    g_99 = f_round(g_c[indx_99[i_y], i_y, i_bs, i_c]./y_full)
    
    
    println("")
    println("                Escenario cobre y PIB bajo")    
    println("Deuda bruta prudente ")
    println("                     BP      r     G/Y    A/Y")
    println("Prob 10%:        ", (mean_b90, r_90, g_90, bs_y_p90))
    println("Prob 5% :        ", (mean_b95, r_95, g_95, bs_y_p95))
    println("Prob 1% :        ", (mean_b99, r_99, g_99, bs_y_p99))       
    
    
    """ Escenario intermedio """
    #Cobre y pib
    i_c = Int(floor(Nc/2)+1) 
    i_y = Int(floor(Ny/2)+1)     
    
    #Deuda
    y_full = fun_y(ae, i_y, i_c)
    b_p90, b_p95, b_p99 = get_debt_limit(ae, i_bs, i_c) #
    
    mean_b90 = f_round(mean(b_p90./y_full))
    mean_b95 = f_round(mean(b_p95./y_full))
    mean_b99 = f_round(mean(b_p99./y_full))  
    
    #Índice deuda prudente
    indx_99, indx_95, indx_90 = get_debt_limit(ae, i_bs, i_c, indx=true)    

    #Activos
    bs_y = f_round(grid_bs[i_bs] /y_full)
    
    bs_y_p90 = f_round(bs_c[indx_90[i_y], i_y, i_bs, i_c]./y_full, 5) 
    bs_y_p95 = f_round(bs_c[indx_95[i_y], i_y, i_bs, i_c]./y_full, 5) 
    bs_y_p99 = f_round(bs_c[indx_99[i_y], i_y, i_bs, i_c]./y_full, 5) 
            
    
    
    #Tasa de interés
    r_90 =  f_round((1 ./ q[indx_90[i_y], i_y, i_bs, i_c] .-1 - rf)*10_000)
    r_95 =  f_round((1 ./ q[indx_95[i_y], i_y, i_bs, i_c] .-1 - rf)*10_000)
    r_99 =  f_round((1 ./ q[indx_99[i_y], i_y, i_bs, i_c] .-1 - rf)*10_000)

    # r_90 =  f_round((1 ./ q[indx_90[i_y], i_y, i_bs, i_c] .-1 ))
    # r_95 =  f_round((1 ./ q[indx_95[i_y], i_y, i_bs, i_c] .-1 ))
    # r_99 =  f_round((1 ./ q[indx_99[i_y], i_y, i_bs, i_c] .-1 ))
        
    #Gasto
    g_90 = f_round(g_c[indx_90[i_y], i_y, i_bs, i_c]./y_full)
    g_95 = f_round(g_c[indx_95[i_y], i_y, i_bs, i_c]./y_full)
    g_99 = f_round(g_c[indx_99[i_y], i_y, i_bs, i_c]./y_full)
    
    
    println("")
    println("                Escenario cobre y PIB intermedio")    
    println("Deuda bruta prudente ")
    println("                     BP      r     G/Y    A/Y")
    println("Prob 10%:        ", (mean_b90, r_90, g_90, bs_y_p90))
    println("Prob 5% :        ", (mean_b95, r_95, g_95, bs_y_p95))
    println("Prob 1% :        ", (mean_b99, r_99, g_99, bs_y_p99))    
        
    
    """ Escenario alto """
    #Cobre y PIB
    if extremo == false    
        i_c = Int(floor(Nc*0.9)+1) 
        i_y = Int(floor(Ny*0.9)+1) 
    else
        i_c = Int(Nc) 
        i_y = Int(Ny)
    end
     
    
    #Deuda
    y_full = fun_y(ae, i_y, i_c)
    b_p90, b_p95, b_p99 = get_debt_limit(ae, i_bs, i_c) #
    
    mean_b90 = f_round(mean(b_p90./y_full))
    mean_b95 = f_round(mean(b_p95./y_full))
    mean_b99 = f_round(mean(b_p99./y_full))  
    

    #Índice deuda prudente
    indx_99, indx_95, indx_90 = get_debt_limit(ae, i_bs, i_c, indx=true)    

    #Activos
    bs_y = f_round(grid_bs[i_bs] /y_full)
    bs_y_p90 = f_round(bs_c[indx_90[i_y], i_y, i_bs, i_c]./y_full, 5) 
    bs_y_p95 = f_round(bs_c[indx_95[i_y], i_y, i_bs, i_c]./y_full, 5) 
    bs_y_p99 = f_round(bs_c[indx_99[i_y], i_y, i_bs, i_c]./y_full, 5)     
    
    #Tasa de interés
    r_90 =  f_round((1 ./ q[indx_90[i_y], i_y, i_bs, i_c] .-1 - rf)*10_000)
    r_95 =  f_round((1 ./ q[indx_95[i_y], i_y, i_bs, i_c] .-1 - rf)*10_000)
    r_99 =  f_round((1 ./ q[indx_99[i_y], i_y, i_bs, i_c] .-1 - rf)*10_000)
    
    # r_90 =  f_round((1 ./ q[indx_90[i_y], i_y, i_bs, i_c] .-1 ))
    # r_95 =  f_round((1 ./ q[indx_95[i_y], i_y, i_bs, i_c] .-1 ))
    # r_99 =  f_round((1 ./ q[indx_99[i_y], i_y, i_bs, i_c] .-1 ))
        
    #Gasto
    g_90 = f_round(g_c[indx_90[i_y], i_y, i_bs, i_c]./y_full)
    g_95 = f_round(g_c[indx_95[i_y], i_y, i_bs, i_c]./y_full)
    g_99 = f_round(g_c[indx_99[i_y], i_y, i_bs, i_c]./y_full)
    
    
    println("")
    println("                Escenario cobre y PIB alto")    
    println("Deuda bruta prudente ")
    println("                     BP      r     G/Y    A/Y")
    println("Prob 10%:        ", (mean_b90, r_90, g_90, bs_y_p90))
    println("Prob 5% :        ", (mean_b95, r_95, g_95, bs_y_p95))
    println("Prob 1% :        ", (mean_b99, r_99, g_99, bs_y_p99))    
        
    
end

In [ ]:
function table_deuda_bruta_3(ae; i_i=2)
    @unpack Nc, Nbs, Ny, q, g_c, grid_bs, rf, bs_c, Ni = ae
    
    #Funciones auxuliares
    f_round(x,d=4) = round.(x, digits=d)
    

    B, BP, BSP, Q, G, escenario, Yfull, YNC, YC, BS, Prob = get_debt_limit2(ae) #     
    list_pr = range(0.00, 0.1, length=Ni)
    R = 10_000*(1 ./Q .-1 .- rf)
    
    if i_i==11
        ϵ = 0.0001#1
    else
        ϵ = 0.01#5
    end
    
    i_B = findall(x -> x < median(B[i_i, :,:,:]).+ϵ && x >= median(B[i_i, :,:,:]), B[i_i,:,:,:])
    
    
    prob = Int(round(100*(1-list_pr[i_i]), digits=0))
    m_B  = B[i_i,:,:,:]
    m_BP = BP[i_i,:,:,:]
    m_BS = BS[i_i,:,:,:]
    m_BSP = BSP[i_i,:,:,:]
    m_R  = R[i_i,:,:,:]
    m_G  = G[i_i, :,:,:]
    m_esc = escenario[i_i, :,:,:]
    
    m_Yf  = Yfull[i_i,:,:,:]
    m_YNC = YNC[i_i, :,:,:]
    m_YC  = YC[i_i, :,:,:]    
    m_Pr  = Prob[i_i, :,:,:]    
    
    function mean_med(x)
        return (f_round(mean(x)), f_round(median(x)))
    end
    
    println("Probabilidad default = ", i_i-1, "%")
    println("")
    println("B/Y:       ", f_round(m_B[i_B]), " mean, med: ", mean_med(m_B[i_B]))
    println("BP/Y:      ", f_round(m_BP[i_B]), " mean, med: ", mean_med(m_BP[i_B]))
    println("BS/Y:      ", f_round(m_BS[i_B]), " mean:, med ", mean_med(m_BS[i_B]))
    println("BSP/Y:     ", f_round(m_BSP[i_B]), " mean, med: ", mean_med(m_BSP[i_B]))
    println("Spread:    ", f_round(m_R[i_B]), " mean, med: ", mean_med(m_R[i_B]))
    println("G/Y:       ", f_round(m_G[i_B]), " mean, med: ", mean_med(m_G[i_B]))
    println("Escenario: ", m_esc[i_B])
    println("Yfull:     ", f_round(m_Yf[i_B]), " mean, med: ", mean_med(m_Yf[i_B]))
    println("YNC:       ", f_round(m_YNC[i_B]), " mean, med: ", mean_med(m_YNC[i_B]))
    println("YC:        ", f_round(m_YC[i_B]), " mean, med: ", mean_med(m_YC[i_B]))
    println("Prob:      ", f_round(m_Pr[i_B]), " mean, med: ", mean_med(m_Pr[i_B]))
   
end

In [ ]:
function table_deuda_bruta_2(ae; table_escenario=false, mediana=true)
    @unpack Nc, Nbs, Ny, q, g_c, grid_bs, rf, bs_c, Ni = ae
    
    #Funciones auxuliares
    f_round(x,d=3) = round(x, digits=d)
    

    B, BP, BSP, Q, G, escenario, Yfull, YNC, YC, BS, Prob   = get_debt_limit2(ae) #     
    list_pr = range(0.00, 0.1, length=Ni)
    R = 10_000*(1 ./Q .-1 .- rf)
    
    # print("ACA")
    println("")
    println("                Escenario cobre y PIB intermedio")    
    println("Deuda bruta prudente ")
    println("                     B/Y    BP/Y   BS/Y spread   G/Y ")

    if mediana==true
        if table_escenario == false
            for i_i in 1:Ni
                prob = Int(round(100*(1-list_pr[i_i]), digits=0))
                m_B  = f_round(median(B[i_i, :,:,:]))
                m_BP = f_round(median(BP[i_i,:,:,:]))
                m_BS = f_round(median(BSP[i_i,:,:,:]))
                m_R  = f_round(median(R[i_i,:,:,:]),1)
                m_G  = f_round(median(G[i_i, :,:,:]))
                println("Prob :", prob, "        ",  (m_B, m_BP, m_BS, m_R, m_G))
            end

        else

            m_B_A = zeros(Ni); m_B_B = zeros(Ni); m_B_I = zeros(Ni)
            m_BP_A = zeros(Ni); m_BP_B = zeros(Ni); m_BP_I = zeros(Ni)
            m_BS_A = zeros(Ni); m_BS_B = zeros(Ni); m_BS_I = zeros(Ni)
            m_R_A = zeros(Ni); m_R_B = zeros(Ni); m_R_I = zeros(Ni)
            m_G_A = zeros(Ni); m_G_B = zeros(Ni); m_G_I = zeros(Ni)

            for i_i in 1:Ni
                #Posición escenarios
                esc_adv = findall(x->x==0, escenario[i_i,:,:,:][:])
                esc_bue = findall(x->x==1, escenario[i_i,:,:,:][:])
                esc_int = findall(x->x==2, escenario[i_i,:,:,:][:])

                #Deuda presente
                m_B_A[i_i] = f_round(median(B[i_i, :,:,:][:][esc_adv]))
                m_B_B[i_i] = f_round(median(B[i_i, :,:,:][:][esc_bue]))
                m_B_I[i_i] = f_round(median(B[i_i, :,:,:][:][esc_int]))

                #Deuda futura
                m_BP_A[i_i] = f_round(median(BP[i_i, :,:,:][:][esc_adv]))
                m_BP_B[i_i] = f_round(median(BP[i_i, :,:,:][:][esc_bue]))
                m_BP_I[i_i] = f_round(median(BP[i_i, :,:,:][:][esc_int]))

                #Activos
                m_BS_A[i_i] = f_round(median(BSP[i_i, :,:,:][:][esc_adv]))
                m_BS_B[i_i] = f_round(median(BSP[i_i, :,:,:][:][esc_bue]))
                m_BS_I[i_i] = f_round(median(BSP[i_i, :,:,:][:][esc_int]))

                #Spread
                m_R_A[i_i] = f_round(median(R[i_i, :,:,:][:][esc_adv]))
                m_R_B[i_i] = f_round(median(R[i_i, :,:,:][:][esc_bue]))
                m_R_I[i_i] = f_round(median(R[i_i, :,:,:][:][esc_int]))

                #Gasto
                m_G_A[i_i] = f_round(median(G[i_i, :,:,:][:][esc_adv]))
                m_G_B[i_i] = f_round(median(G[i_i, :,:,:][:][esc_bue]))
                m_G_I[i_i] = f_round(median(G[i_i, :,:,:][:][esc_int]))
                
                end

            println("Escenario adverso")
            for i_i in 1:Ni
                prob = Int(round(100*(1-list_pr[i_i]), digits=0))
                println("Prob :", prob, "        ",  (m_B_A[i_i], m_BP_A[i_i], m_BS_A[i_i], m_R_A[i_i], m_G_A[i_i]))
            end

            println("Escenario Intermedio")
            for i_i in 1:Ni
                prob = Int(round(100*(1-list_pr[i_i]), digits=0))
                println("Prob :", prob, "        ",  (m_B_I[i_i], m_BP_I[i_i], m_BS_I[i_i], m_R_I[i_i], m_G_I[i_i]))
            end

            println("Escenario Bueno")
            for i_i in 1:Ni
                prob = Int(round(100*(1-list_pr[i_i]), digits=0))
                println("Prob :", prob, "        ",  (m_B_B[i_i], m_BP_B[i_i], m_BS_B[i_i], m_R_B[i_i], m_G_B[i_i]))
            end        
              
        end
    else
        if table_escenario == false
            for i_i in 1:Ni
                prob = Int(round(100*(1-list_pr[i_i]), digits=0))
                m_B  = f_round(mean(B[i_i, :,:,:]))
                m_BP = f_round(mean(BP[i_i,:,:,:]))
                m_BS = f_round(mean(BSP[i_i,:,:,:]))
                m_R  = f_round(mean(R[i_i,:,:,:]),1)
                m_G  = f_round(mean(G[i_i, :,:,:]))
                println("Prob :", prob, "        ",  (m_B, m_BP, m_BS, m_R, m_G))
            end
        
        else
        
            m_B_A = zeros(Ni); m_B_B = zeros(Ni); m_B_I = zeros(Ni)
            m_BP_A = zeros(Ni); m_BP_B = zeros(Ni); m_BP_I = zeros(Ni)
            m_BS_A = zeros(Ni); m_BS_B = zeros(Ni); m_BS_I = zeros(Ni)
            m_R_A = zeros(Ni); m_R_B = zeros(Ni); m_R_I = zeros(Ni)
            m_G_A = zeros(Ni); m_G_B = zeros(Ni); m_G_I = zeros(Ni)
        
            for i_i in 1:Ni
                #Posición escenarios
                esc_adv = findall(x->x==0, escenario[i_i,:,:,:][:])
                esc_bue = findall(x->x==1, escenario[i_i,:,:,:][:])
                esc_int = findall(x->x==2, escenario[i_i,:,:,:][:])

                #Deuda presente
                m_B_A[i_i] = f_round(mean(B[i_i, :,:,:][:][esc_adv]))
                m_B_B[i_i] = f_round(mean(B[i_i, :,:,:][:][esc_bue]))
                m_B_I[i_i] = f_round(mean(B[i_i, :,:,:][:][esc_int]))

                #Deuda futura
                m_BP_A[i_i] = f_round(mean(BP[i_i, :,:,:][:][esc_adv]))
                m_BP_B[i_i] = f_round(mean(BP[i_i, :,:,:][:][esc_bue]))
                m_BP_I[i_i] = f_round(mean(BP[i_i, :,:,:][:][esc_int]))

                #Activos
                m_BS_A[i_i] = f_round(mean(BSP[i_i, :,:,:][:][esc_adv]))
                m_BS_B[i_i] = f_round(mean(BSP[i_i, :,:,:][:][esc_bue]))
                m_BS_I[i_i] = f_round(mean(BSP[i_i, :,:,:][:][esc_int]))

                #Spread
                m_R_A[i_i] = f_round(mean(R[i_i, :,:,:][:][esc_adv]))
                m_R_B[i_i] = f_round(mean(R[i_i, :,:,:][:][esc_bue]))
                m_R_I[i_i] = f_round(mean(R[i_i, :,:,:][:][esc_int]))

                #Gasto
                m_G_A[i_i] = f_round(mean(G[i_i, :,:,:][:][esc_adv]))
                m_G_B[i_i] = f_round(mean(G[i_i, :,:,:][:][esc_bue]))
                m_G_I[i_i] = f_round(mean(G[i_i, :,:,:][:][esc_int]))
                
                println("Escenario adverso")
                for i_i in 1:Ni
                    prob = Int(round(100*(1-list_pr[i_i]), digits=0))
                    println("Prob :", prob, "        ",  (m_B_A[i_i], m_BP_A[i_i], m_BS_A[i_i], m_R_A[i_i], m_G_A[i_i]))
                end

                println("Escenario Intermedio")
                for i_i in 1:Ni
                    prob = Int(round(100*(1-list_pr[i_i]), digits=0))
                    println("Prob :", prob, "        ",  (m_B_I[i_i], m_BP_I[i_i], m_BS_I[i_i], m_R_I[i_i], m_G_I[i_i]))
                end

                println("Escenario Bueno")
                for i_i in 1:Ni
                    prob = Int(round(100*(1-list_pr[i_i]), digits=0))
                    println("Prob :", prob, "        ",  (m_B_B[i_i], m_BP_B[i_i], m_BS_B[i_i], m_R_B[i_i], m_G_B[i_i]))
                end        
            end                
        end
    end
end